In [1]:
from keras.layers import Dense, Dropout, Activation, Flatten
import numpy as np

Using TensorFlow backend.


In [51]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Cropping2D
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.layers.core import Lambda,Dropout

In [3]:
# took some help to define LeNet using keras from http://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/
def LeNet(width,height,depth):
    model = Sequential()
    #normalization
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(width,height,depth)))
    #first conv +pooling
    model.add(Convolution2D(6,5,5,input_shape=(width,height,depth),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #second conv+pooling          
    model.add(Convolution2D(16,5,5,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #convolutions have ended. begin flattening and fully connected layers
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Activation('relu'))
    #
    model.add(Dense(84))
    model.add(Activation('relu'))
    #
    model.add(Dense(1))
    return model


In [72]:
def nvidia_net(width,height,depth):
    model=Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(width,height,depth)))
    model.add(Cropping2D(cropping=((50,20),(0,0)),input_shape=(width,height,depth)))
    
     #first conv
    model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
    #second conv         
    model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
    #third conv
    model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
    #fourth conv 3x3
    model.add(Convolution2D(64,3,3,activation='relu'))
    #fifth conv 3x3
    model.add(Convolution2D(64,3,3,activation='relu'))
    #convolutions have ended. begin flattening and fully connected layers
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(100))
    model.add(Activation('relu'))
    #
    model.add(Dropout(0.2))
    model.add(Dense(50))
    model.add(Activation('relu'))
    #
    model.add(Dense(10))
    model.add(Activation('relu'))
    #
    model.add(Dense(1))
    return model
    

In [4]:
def preprocess_driving_log(driving_log):
    #only keep paths after IMG. the driving log generated by simulator is full path on windows machine while I am training on AWS
    def trim_til_IMG(str):
        components = str.split('\\')    #driving log was generated on windows machine hence the backslash
        if len(components)<2:  #linux driving log..do nothing
            return str
        else:
            return (components[-2]+'/'+components[-1])
    driving_log[['center','left','right']] = driving_log[['center','left','right']].applymap(trim_til_IMG)
    return driving_log

In [54]:
#data stuff
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from itertools import chain

def dir_to_df(dir):
    driving_log = shuffle(pd.read_csv(dir+'/driving_log.csv',header=None))
    driving_log.columns=['center','left','right','steering','throttle','break','speed']
    driving_log = preprocess_driving_log(driving_log)
    return driving_log

def train_validation_split(dir='/home/carnd/data'):
    driving_log = dir_to_df(dir)
    
    training_df,validation_df = train_test_split(driving_log,test_size=0.1)
    return training_df,validation_df

def process_row2(row,dir):
    row=row[1]
    center_image = cv2.imread(dir +'/' + row['center'])
    left_image = cv2.imread(dir+'/'+row['left'].strip())
    right_image = cv2.imread(dir+'/'+row['right'].strip())
    correction = 0.20
    steering =row['steering']
    return_value = [left_image, center_image, right_image],[steering+correction , steering, steering-correction]
    return return_value

def process_row1(row,dir):
    row=row[1]
    image = cv2.imread(dir +'/' + row['center'])
    steering = row['steering']
    return([image],[steering])# returning list here to by type compatible with process_row2

    #img_path_list = driving_log['center'].map(lambda x:dir+'/'+x).tolist()
    #img_list = np.array([cv2.imread(img_path) for img_path in img_path_list])
    #steering_direction = driving_log['steering'].values
    #img_path_model_list,img_path_validation_list,y_model,y_validation = train_test_split(img_path_list,
                                   # steering_direction,test_size=0.1,random_state=100)
    #return (img_path_model_list,img_path_validation_list,y_model,y_validation)

    
    
def generator(training_df, batch_size=32,dir='/home/carnd/data'):
    while(1):
       for i in range(0,len(training_df),batch_size):
            batch_data = [process_row2(j,dir) for j in training_df.iloc[i:i+batch_size].iterrows()]
            batch_image_data = list(chain.from_iterable([row_data[0] for row_data in batch_data]))
            batch_steering_data = list(chain.from_iterable([row_data[1] for row_data in batch_data]))
            yield shuffle(np.array(batch_image_data),np.array(batch_steering_data))
        

In [18]:
training_df.head()

,center,left,right,steering,throttle,break,speed
4793,IMG/center_2016_12_01_13_41_12_957.jpg,IMG/left_2016_12_01_13_41_12_957.jpg,IMG/right_2016_12_01_13_41_12_957.jpg,-0.097735,0.985533,0.0,30.18421
5890,IMG/center_2016_12_01_13_43_04_188.jpg,IMG/left_2016_12_01_13_43_04_188.jpg,IMG/right_2016_12_01_13_43_04_188.jpg,-0.145206,0.985533,0.0,30.18584
6857,IMG/center_2016_12_01_13_44_42_236.jpg,IMG/left_2016_12_01_13_44_42_236.jpg,IMG/right_2016_12_01_13_44_42_236.jpg,-0.097735,0.985533,0.0,30.18622
5743,IMG/center_2016_12_01_13_42_49_264.jpg,IMG/left_2016_12_01_13_42_49_264.jpg,IMG/right_2016_12_01_13_42_49_264.jpg,0.000000,0.985533,0.0,30.18664
3980,IMG/center_2016_12_01_13_39_21_657.jpg,IMG/left_2016_12_01_13_39_21_657.jpg,IMG/right_2016_12_01_13_39_21_657.jpg,0.000000,0.534023,0.0,13.06413


In [41]:

#type(steering_direction)

In [73]:
training_df,validation_df = train_validation_split('/home/carnd/data')
training_data_generator = generator(training_df)
validation_data_generator = generator(validation_df)

In [74]:
model = nvidia_net(160,320,3)
model.compile(loss='mse',optimizer='adam')

model.fit_generator(training_data_generator,samples_per_epoch=3*len(training_df),validation_data=validation_data_generator,nb_epoch=2,nb_val_samples=3*len(validation_df),verbose=0)

In [42]:
model.save('/home/carnd/behavioral-cloning/nvidia_generator_left_right.h5')

In [69]:
#some more training on additional data generated using the simulator
back_to_center_df = dir_to_df('/home/carnd/driving-data/back-to-centre')
normal_df = dir_to_df('/home/carnd/driving-data/normal')
reverse_lap_df = dir_to_df('/home/carnd/driving-data/reverse-lap')
track_2_df = dir_to_df('/home/carnd/driving-data/track-2')

In [75]:
model.fit_generator(generator(back_to_center_df,batch_size=32,dir='/home/carnd/driving-data/back-to-centre'),samples_per_epoch=3*len(back_to_center_df),validation_data=validation_data_generator,nb_epoch=2,nb_val_samples=3*len(validation_df),verbose=0)
model.fit_generator(generator(track_2_df,batch_size=32,dir='/home/carnd/driving-data/track-2'),samples_per_epoch=3*len(track_2_df),validation_data=validation_data_generator,nb_epoch=2,nb_val_samples=3*len(validation_df),verbose=0)
model.fit_generator(generator(normal_df,batch_size=32,dir='/home/carnd/driving-data/normal'),samples_per_epoch=3*len(normal_df),validation_data=validation_data_generator,nb_epoch=2,nb_val_samples=3*len(validation_df),verbose=0)
model.fit_generator(generator(reverse_lap_df,batch_size=32,dir='/home/carnd/driving-data/reverse-lap'),samples_per_epoch=3*len(reverse_lap_df),validation_data=validation_data_generator,nb_epoch=2,nb_val_samples=3*len(validation_df),verbose=0)

In [76]:
model.save('/home/carnd/behavioral-cloning/nvidia_generator_all_data_cropping_dropout.h5')

In [32]:
t

NameError: name 't' is not defined

In [27]:
model.save('/home/carnd/behavioral-cloning/model_generator_all_data.h5')

In [40]:
import pandas as pd
a=pd.DataFrame(columns=['a','b'])

In [41]:
a['a']=np.random.randint(10,20,10)
a['b']=np.random.randint(10,20,10)

In [44]:
[row[1]['a'] for row in a.iterrows()]

[14, 19, 12, 15, 15, 12, 13, 19, 12, 15]

In [19]:
from sklearn.utils import shuffle
shuffle(a)

,a,b
3,18,13
8,17,18
1,12,19
5,13,17
0,18,17
6,16,10
4,15,17
9,19,15
7,15,18
2,10,15


In [9]:
for i in range(0,10,3):
   print(list(range(10))[i:i+3])

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9]


In [11]:
list(range(10))[1:12]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [23]:
[print(i) for i in zip([1,2,4],[4,5,6])]

(1, 4)
(2, 5)
(4, 6)


[None, None, None]

In [31]:
len(v)

804

In [74]:
a=generator(v.iloc[4:10],batch_size=2)
i=0
for j in a:
    print(j[0].shape)
    i +=1
    if i>2:
        break

2
/home/carnd/data/ IMG/left_2016_12_01_13_45_49_693.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
2
/home/carnd/data/ IMG/left_2016_12_01_13_36_26_502.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(6, 160, 320, 3)
2
/home/carnd/data/ IMG/left_2016_12_01_13_32_52_753.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
2
/home/carnd/data/ IMG/left_2016_12_01_13_39_13_161.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(6, 160, 320, 3)
2
/home/carnd/data/ IMG/left_2016_12_01_13_45_36_396.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
2
/home/carnd/data/ IMG/left_2016_12_01_13_37_31_162.jpg
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(6, 160, 320, 3)


In [52]:
t.head()

,0,1,2,3,4,5,6
1165,IMG/center_2016_12_01_13_34_36_337.jpg,IMG/left_2016_12_01_13_34_36_337.jpg,IMG/right_2016_12_01_13_34_36_337.jpg,0.000000,0.985533,0.0,30.18665
2763,IMG/center_2016_12_01_13_37_18_295.jpg,IMG/left_2016_12_01_13_37_18_295.jpg,IMG/right_2016_12_01_13_37_18_295.jpg,0.176582,0.985533,0.0,30.18601
7416,IMG/center_2016_12_01_13_45_38_931.jpg,IMG/left_2016_12_01_13_45_38_931.jpg,IMG/right_2016_12_01_13_45_38_931.jpg,0.272268,0.985533,0.0,30.18008
1010,IMG/center_2016_12_01_13_34_20_600.jpg,IMG/left_2016_12_01_13_34_20_600.jpg,IMG/right_2016_12_01_13_34_20_600.jpg,0.176582,0.985533,0.0,30.18553
465,IMG/center_2016_12_01_13_33_25_348.jpg,IMG/left_2016_12_01_13_33_25_348.jpg,IMG/right_2016_12_01_13_33_25_348.jpg,0.090465,0.985533,0.0,30.18618
